# Homomorphic Encrypted LeNet-1
This notebook will show a very practical example of running the famous LeNet-1 DL model directly on encrypted data.

![scheme](HE_processing.png)

## Homomorphic encryption operations
First of all, we will look at Pyfhel, a Python library which wraps SEAL, one of the most used frameworks for HE.
Pyfhel supports the BFV scheme, so, it is the one that we will use.

In [1]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt

HE = Pyfhel()
HE.contextGen(p=65537, m=4096)
HE.keyGen()

print(HE)

a = 127.15717263
b = -2.128965182
ctxt1 = HE.encryptFrac(a)
ctxt2 = HE.encryptFrac(b)

ctxtSum = ctxt1 + ctxt2
ctxtSub = ctxt1 - ctxt2
ctxtMul = ctxt1 * ctxt2

resSum = HE.decryptFrac(ctxtSum)
resSub = HE.decryptFrac(ctxtSub) 
resMul = HE.decryptFrac(ctxtMul)

print(f"Expected sum: {a+b}, decrypted sum: {resSum}")
print(f"Expected sub: {a-b}, decrypted sum: {resSub}")
print(f"Expected mul: {a*b}, decrypted sum: {resMul}")

<Pyfhel obj at 0x7efe592d9bd0, [pk:Y, sk:Y, rtk:-, rlk:-, contx(p=65537, m=4096, base=2, sec=128, dig=64i.32f, batch=False)]>
Expected sum: 125.028207448, decrypted sum: 125.02820744784549
Expected sub: 129.286137812, decrypted sum: 129.28613781183958
Expected mul: -270.7131931708334, decrypted sum: -270.7131931686308


In [6]:
m1 = HE.encryptFrac(a)
print(HE.noiseLevel(m1))

m2 = HE.encodeFrac(2)

82


In [7]:
print(HE.noiseLevel(m1+m1))
print(HE.noiseLevel(m1*m1))
print(HE.noiseLevel(m1+m2))
print(HE.noiseLevel(m1*m2))

81
54
82
82


Before starting, let's note that:
  1. We will use the fractional encoder to encode (and encrypt) the values in our examples. BFV was born for integers, so, CKKS should be used if the use case involves fractional values. However it is a more complex scheme, and for this example BFV is sufficient.
  2. We will not use batching (also called *packing*). While batching can greatly speed up the computations, it introduces limitations which make the encrypted ML much more complex. For this example, we will encrypt/encode each number with a polynomial.

## LeNet-1
The LeNet-1 is a small CNN developed by LeCun et al. It is composed of 5 layers: a convolutional layer with 4 kernels of size 5x5 and tanh activation, an average pooling layer with kernel of size 2, another convolutional layer with 16 kernels of size 5x5 and tanh activation, another average pooling layer with kernel of size 2, and a fully connected layers with size 192x10. 

The highest value in the output tensor corresponds to the label LeNet-1 associated to the input image. 

For this tutorial we will use the MNIST dataset.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root = './data',
    train=True,
    download=True,
    transform=transform
)

test_set = torchvision.datasets.MNIST(
    root = './data',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=50,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=50,
    shuffle=True
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



102.8%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



112.7%
/home/ale/DatiLinux/Repos/Pyfhel/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


In [13]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

def train_net(network, epochs, device):
    optimizer = optim.Adam(network.parameters(), lr=0.001)
    for epoch in range(epochs):

        total_loss = 0
        total_correct = 0

        for batch in train_loader: # Get Batch
            images, labels = batch 
            images, labels = images.to(device), labels.to(device)

            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)

        
def test_net(network, device):
    network.eval()
    total_loss = 0
    total_correct = 0
    
    with torch.no_grad():
        for batch in test_loader: # Get Batch
            images, labels = batch 
            images, labels = images.to(device), labels.to(device)

            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)

        accuracy = round(100. * (total_correct / len(test_loader.dataset)), 4)

    return total_correct / len(test_loader.dataset)

In [14]:
train = True # If set to false, it will load models previously trained and saved.

In [15]:
experiments = 10

In [16]:
if train:
    accuracies = []
    for i in range(0, experiments):
        LeNet1 = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),

            nn.Conv2d(4, 12, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),

            nn.Flatten(),

            nn.Linear(192, 10),
        )
        
        LeNet1.to(device)
        train_net(LeNet1, 15, device)
        acc = test_net(LeNet1, device)
        accuracies.append(acc)
#         torch.save(LeNet1, "LeNet1.pt")
else:
    LeNet1 = torch.load("LeNet1.pt")
    LeNet1.eval()
    LeNet1.to(device)

In [24]:
m = np.array(accuracies)
print(f"Mean accuracy on test set: {np.mean(m)}")
print(f"Var: {np.var(m)}")

Mean: 0.9882700000000002
Var: 9.140999999999878e-07


## Approximating
As we know, there are some operations that cannot be performed homomorphically on encrypted values. Most notably, these operations are division and comparison. It is possible to perform only linear functions.

Consequently, in the LeNet-1 scheme we used, we can not use `tanh()`. This is because we cannot apply its non-linearities.


One of the most common approach is to replace it with a simple polynomial function, for example a square layer (which simply performs $x \rightarrow x^2$).

We define the model with all the non-linearities removed **approximated**. This model can be re-trained, and it will be ready to be used on encrypted values.

In [18]:
class Square(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, t):
        return torch.pow(t, 2)

LeNet1_Approx = nn.Sequential(
    nn.Conv2d(1, 4, kernel_size=5),
    Square(),
    nn.AvgPool2d(kernel_size=2),
            
    nn.Conv2d(4, 12, kernel_size=5),
    Square(),
    nn.AvgPool2d(kernel_size=2),
    
    nn.Flatten(),
    
    nn.Linear(192, 10),
)

In [19]:
if train:
    approx_accuracies = []
    for i in range(0, experiments):
        LeNet1_Approx = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5),
            Square(),
            nn.AvgPool2d(kernel_size=2),

            nn.Conv2d(4, 12, kernel_size=5),
            Square(),
            nn.AvgPool2d(kernel_size=2),

            nn.Flatten(),

            nn.Linear(192, 10),
        )
        
        LeNet1_Approx.to(device)
        train_net(LeNet1_Approx, 15, device)
        acc = test_net(LeNet1_Approx, device)
        approx_accuracies.append(acc)
        torch.save(LeNet1, "LeNet1_Approx.pt")

else:
    LeNet1_Approx = torch.load("LeNet1_Approx.pt")
    LeNet1_Approx.eval()
    LeNet1_Approx.to(device)

In [25]:
m = np.array(approx_accuracies)
print(f"Mean: {np.mean(m)}")
print(f"Var: {np.var(m)}")

Mean: 0.98773
Var: 2.024099999999944e-06


We can see that replacing `tanh()` with `square()` did not impact the accuracy of the model dramatically. Usually this is not the case, and approximating DL models may worsen the performance badly. This is one of the challenges that HE-ML will have to consider: the creation of DL models keeping in mind the HE constraints from the beginning.

In any case, now the network is HE-compatible.

## Encoding
From the applicative point of view, we have two options on how we want our Torch model to run on encrypted values:
  1. Modify Torch layers code in order to be fully compatible with arrays of Pyfhel ciphertexts/encoded values;
  2. Create the code for the general blocks of LeNet-1 (convolutional layer, linear layer, square layer, flatten...)
  
We opt for the second path, having already done this in our previous work: https://github.com/AlexMV12/PyCrCNN

Let's remember that, in order to be used with the encrypted values, also the weights of the models will have to be **encoded**. This means that each value in the weights of each layer will be encoded in a polynomial.

First, we define some useful functions to encrypt/encode matrices:

In [13]:
def encode_matrix(HE, matrix):
    try:
        return np.array(list(map(HE.encodeFrac, matrix)))
    except TypeError:
        return np.array([encode_matrix(HE, m) for m in matrix])


def decode_matrix(HE, matrix):
    try:
        return np.array(list(map(HE.decodeFrac, matrix)))
    except TypeError:
        return np.array([decode_matrix(HE, m) for m in matrix])


def encrypt_matrix(HE, matrix):
    try:
        return np.array(list(map(HE.encryptFrac, matrix)))
    except TypeError:
        return np.array([encrypt_matrix(HE, m) for m in matrix])


def decrypt_matrix(HE, matrix):
    try:
        return np.array(list(map(HE.decryptFrac, matrix)))
    except TypeError:
        return np.array([decrypt_matrix(HE, m) for m in matrix])

Then, the actual code for the convolutional, linear, square, flatten and average pooling layer is required:

In [14]:
class ConvolutionalLayer:
    def __init__(self, HE, weights, stride=(1, 1), padding=(0, 0), bias=None):
        self.HE = HE
        self.weights = encode_matrix(HE, weights)
        self.stride = stride
        self.padding = padding
        self.bias = bias
        if bias is not None:
            self.bias = encode_matrix(HE, bias)

    def __call__(self, t):
        t = apply_padding(t, self.padding)
        result = np.array([[np.sum([convolute2d(image_layer, filter_layer, self.stride)
                                    for image_layer, filter_layer in zip(image, _filter)], axis=0)
                            for _filter in self.weights]
                           for image in t])

        if self.bias is not None:
            return np.array([[layer + bias for layer, bias in zip(image, self.bias)] for image in result])
        else:
            return result


def convolute2d(image, filter_matrix, stride):
    x_d = len(image[0])
    y_d = len(image)
    x_f = len(filter_matrix[0])
    y_f = len(filter_matrix)

    y_stride = stride[0]
    x_stride = stride[1]

    x_o = ((x_d - x_f) // x_stride) + 1
    y_o = ((y_d - y_f) // y_stride) + 1

    def get_submatrix(matrix, x, y):
        index_row = y * y_stride
        index_column = x * x_stride
        return matrix[index_row: index_row + y_f, index_column: index_column + x_f]

    return np.array(
        [[np.sum(get_submatrix(image, x, y) * filter_matrix) for x in range(0, x_o)] for y in range(0, y_o)])

def apply_padding(t, padding):
    y_p = padding[0]
    x_p = padding[1]
    zero = t[0][0][y_p+1][x_p+1] - t[0][0][y_p+1][x_p+1]
    return [[np.pad(mat, ((y_p, y_p), (x_p, x_p)), 'constant', constant_values=zero) for mat in layer] for layer in t]

In [15]:
class LinearLayer:
    def __init__(self, HE, weights, bias=None):
        self.HE = HE
        self.weights = encode_matrix(HE, weights)
        self.bias = bias
        if bias is not None:
            self.bias = encode_matrix(HE, bias)

    def __call__(self, t):
        result = np.array([[np.sum(image * row) for row in self.weights] for image in t])
        if self.bias is not None:
            result = np.array([row + self.bias for row in result])
        return result

In [16]:
class SquareLayer:
    def __init__(self, HE):
        self.HE = HE

    def __call__(self, image):
        return square(self.HE, image)


def square(HE, image):
    try:
        return np.array(list(map(lambda x: HE.power(x, 2), image)))
    except TypeError:
        return np.array([square(HE, m) for m in image])

In [17]:
class FlattenLayer:
    def __call__(self, image):
        dimension = image.shape
        return image.reshape(dimension[0], dimension[1]*dimension[2]*dimension[3])

In [18]:
class AveragePoolLayer:
    def __init__(self, HE, kernel_size, stride=(1, 1), padding=(0, 0)):
        self.HE = HE
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def __call__(self, t):
        t = apply_padding(t, self.padding)
        return np.array([[_avg(self.HE, layer, self.kernel_size, self.stride) for layer in image] for image in t])


def _avg(HE, image, kernel_size, stride):
    x_s = stride[1]
    y_s = stride[0]

    x_k = kernel_size[1]
    y_k = kernel_size[0]

    x_d = len(image[0])
    y_d = len(image)

    x_o = ((x_d - x_k) // x_s) + 1
    y_o = ((y_d - y_k) // y_s) + 1

    denominator = HE.encodeFrac(1 / (x_k * y_k))

    def get_submatrix(matrix, x, y):
        index_row = y * y_s
        index_column = x * x_s
        return matrix[index_row: index_row + y_k, index_column: index_column + x_k]

    return [[np.sum(get_submatrix(image, x, y)) * denominator for x in range(0, x_o)] for y in range(0, y_o)]

We can now define a function to "convert" a PyTorch model to a list of sequential HE-ready-to-be-used layers:

In [19]:
def build_from_pytorch(HE, net):
    # Define builders for every possible layer

    def conv_layer(layer):
        if layer.bias is None:
            bias = None
        else:
            bias = layer.bias.detach().numpy()

        return ConvolutionalLayer(HE, weights=layer.weight.detach().numpy(),
                                  stride=layer.stride,
                                  padding=layer.padding,
                                  bias=bias)

    def lin_layer(layer):
        if layer.bias is None:
            bias = None
        else:
            bias = layer.bias.detach().numpy()
        return LinearLayer(HE, layer.weight.detach().numpy(),
                           bias)

    def avg_pool_layer(layer):
        # This proxy is required because in PyTorch an AvgPool2d can have kernel_size, stride and padding either of
        # type (int, int) or int, unlike in Conv2d
        kernel_size = (layer.kernel_size, layer.kernel_size) if isinstance(layer.kernel_size, int) else layer.kernel_size
        stride = (layer.stride, layer.stride) if isinstance(layer.stride, int) else layer.stride
        padding = (layer.padding, layer.padding) if isinstance(layer.padding, int) else layer.padding

        return AveragePoolLayer(HE, kernel_size, stride, padding)

    def flatten_layer(layer):
        return FlattenLayer()

    def square_layer(layer):
        return SquareLayer(HE)

    # Maps every PyTorch layer type to the correct builder
    options = {"Conv": conv_layer,
               "Line": lin_layer,
               "Flat": flatten_layer,
               "AvgP": avg_pool_layer,
               "Squa": square_layer
               }

    encoded_layers = [options[str(layer)[0:4]](layer) for layer in net]
    return encoded_layers

## Encrypted processing

Let's list the activities that we will now do:
  1. Create a HE context, specifiying the encryption parameters `m` (polynomial modulus degree) and `p` (plaintext modulus). Let's remember that `q` will be chosen automatically in order to guarantee a 128-bit RSA equivalent security;
  2. Convert our Torch approximated model to a list of layers able to work on matrices of encrypted values. The weights will be encoded;
  3. Encrypt an image from our testing set;
  4. Verify that the final classification result is correct.

If we look at our model, we can see that we have two **square layers**: these are the layers which have more impact on our noise!
Two square layers corresponds to two ciphertext-ciphertext multiplications. Let's see if $m=4096$ gives us enough room to perform 2 encrypted multiplications.

In [20]:
HE = Pyfhel()
HE.contextGen(p=65538, m=4096, fracDigits=64)  # By default, fracDigits=32. Let's increase it to have more precision.
HE.keyGen()
relinKeySize=3
HE.relinKeyGen(bitCount=2, size=relinKeySize)

In [21]:
HE.multDepth(max_depth=64, delta=0.1, x_y_z=(1, 10, 0.1), verbose=True)

Mult 1 [budget: 55 dB]: 10.0 (expected 10)
Mult 2 [budget: 26 dB]: 1.0 (expected 1.0)
Mult 3 [budget: 0 dB]: -4.611686018427388e+18 (expected 10.0)


3

We are near the limit, so we have to change parameters. The simplest thing we can do is to increment $p$ and $m$: this will give us both room for noise and accuracy, at the cost of an heavier computation. Let's try again:

In [22]:
HE = Pyfhel()
HE.contextGen(p=655379848191252, m=8192, fracDigits=64)
HE.keyGen()
relinKeySize=3
HE.relinKeyGen(bitCount=5, size=relinKeySize)

In [23]:
HE.multDepth(max_depth=64, delta=0.1, x_y_z=(1, 10, 0.1), verbose=True)

Mult 1 [budget: 96 dB]: 10.0 (expected 10)
Mult 2 [budget: 32 dB]: 1.0 (expected 1.0)
Mult 3 [budget: 0 dB]: -6.862910432709034e+18 (expected 10.0)


3

Now they should be okay! Let's try to encode our model, encrypt an image and compare the results with the expected ones:

In [24]:
images, labels = next(iter(test_loader))

sample_image = images[0]
sample_label = labels[0]

In [25]:
%%time
LeNet1_Approx.to("cpu")
LeNet1_Encoded = build_from_pytorch(HE, LeNet1_Approx)

CPU times: user 420 ms, sys: 71.7 ms, total: 492 ms
Wall time: 500 ms


What is the expected output?

In [26]:
with torch.no_grad():
    expected_output = LeNet1_Approx(sample_image.unsqueeze(0))

In [27]:
expected_output

tensor([[-12.7048, -10.2085,  10.9441,   4.7339, -29.6800, -23.8598, -14.4046,
         -11.8167,  -2.7060, -13.3006]])

Let's try encrypting the image and passing it through our encoded model.

In [28]:
encrypted_image = encrypt_matrix(HE, sample_image.unsqueeze(0).numpy())

In [29]:
%%time
for layer in LeNet1_Encoded:
    encrypted_image = layer(encrypted_image)
    print(f"Passed layer {layer}...")
    
print("Finished.")

Passed layer <__main__.ConvolutionalLayer object at 0x7fda4d6fc220>...
Passed layer <__main__.SquareLayer object at 0x7fda4d6fc4c0>...
Passed layer <__main__.AveragePoolLayer object at 0x7fda4d6fcdc0>...
Passed layer <__main__.ConvolutionalLayer object at 0x7fda4d701070>...
Passed layer <__main__.SquareLayer object at 0x7fda4c58e580>...
Passed layer <__main__.AveragePoolLayer object at 0x7fda4d7019d0>...
Passed layer <__main__.FlattenLayer object at 0x7fda4d701940>...
Passed layer <__main__.LinearLayer object at 0x7fda4d6fe5b0>...
Finished.
CPU times: user 10min 3s, sys: 1.84 s, total: 10min 5s
Wall time: 10min 8s


### Accuracy
Let's check the result: we can see the difference with respect to the computation in plain.

In [30]:
result = decrypt_matrix(HE, encrypted_image)
print(result)

[[-12.7047981  -10.20841949  10.94171922   4.72890981 -29.67010898
  -23.8592446  -14.40442349 -11.81639633  -2.70542545 -13.29817405]]


In [31]:
difference = expected_output.numpy() - result
print(difference)

[[-1.58617500e-05 -6.42027431e-05  2.38115966e-03  4.95068368e-03
  -9.88179012e-03 -5.91982557e-04 -1.35647673e-04 -2.96978924e-04
  -6.06827379e-04 -2.47368548e-03]]


We are happy with the precision of this result: the difference between the expected output (obtained running the model in plain, on plain data) and the output obtained with encrypted processing is very low.


### Computational load
Obviously, we cannot ignore the huge computational overhead generated by the encrypted processing.

In fact, the processing of one image took about ~10min on a common desktop machine.
The computation has not been parallelized; so, it used only one thread.

While parallelizing allows to speed up the computation, also the occupied memory is a concern: the processing of this image occupied ~5GB of RAM.